In [1]:
import os
import subprocess as sp
from dotenv import load_dotenv
from datetime import datetime
# from tomark import Tomark
from tabulate import tabulate
import pandas as pd
load_dotenv()

True

In [2]:
NAME = os.environ.get("NAME")
API = os.environ.get("WEBHOOK_API")

text="python_test"

In [35]:
message=f'curl -s -d "payload={{\\"username\\":\\"{NAME}\\", \\"text\\":\\"\`\`\`{df.to_markdown()}\`\`\`\\"}}" "{API}"'
# message

In [20]:
e,o = sp.getstatusoutput(message)
e,o

(0, 'ok')

## 함수화

In [4]:
hyperparameter={"EPOCH":10, "LAYERS":100, "DROPOUT": 0.1, "LEARNING_RATE": 0.0001}
hyperparameter

{'EPOCH': 10, 'LAYERS': 100, 'DROPOUT': 0.1, 'LEARNING_RATE': 0.0001}

In [17]:
df.to_markdown()

'|    |   EPOCH |   LAYERS |   DROPOUT |   LEARNING_RATE |\n|---:|--------:|---------:|----------:|----------------:|\n|  0 |      10 |      100 |       0.1 |          0.0001 |'

In [5]:
df = pd.DataFrame([hyperparameter])
tabulate(df, tablefmt='grid').replace('+','\+')

'\\+---\\+----\\+-----\\+-----\\+--------\\+\n| 0 | 10 | 100 | 0.1 | 0.0001 |\n\\+---\\+----\\+-----\\+-----\\+--------\\+'

In [36]:
def slack_post(model, hype_param, val_loss, name=NAME, api=API):
    now = datetime.now().strftime("🕐 (%Y-%m-%d %H:%M:%S)")
    intro=f"[{name}] {now}"
    table_string = pd.DataFrame([hype_param]).to_markdown()
    param_table = f"\`\`\`\n{table_string}\n\`\`\`"
    text=f"[{model}] Validation Loss: {val_loss:.5g}\n{param_table}"
    message=f'curl -s -d "payload={{\\"text\\":\\"{intro}\n{text}\\"}}" "{api}"'
    e,o = sp.getstatusoutput(message)
    if e != 0:
        raise Exception(f"Slack에 메시지를 보내는 과정에서 다음과 같은 에러 발생.\n{o}")
    return e,o,text

In [37]:
e,o,text = slack_post(model="FM", hype_param=hyperparameter, val_loss=0.15)
e,o

(0, 'ok')

In [38]:
text

'[FM] Validation Loss: 0.15\n\\`\\`\\`\n|    |   EPOCH |   LAYERS |   DROPOUT |   LEARNING_RATE |\n|---:|--------:|---------:|----------:|----------------:|\n|  0 |      10 |      100 |       0.1 |          0.0001 |\n\\`\\`\\`'